In [103]:
import tensorflow.compat.v1 as tf
import numpy as np
from tqdm import tqdm_notebook
from sklearn.cluster import AgglomerativeClustering

In [104]:
mnist = tf.keras.datasets.mnist.load_data()

In [105]:
(X_train, _), (X_test, _) = mnist

In [106]:
X_train.shape

(60000, 28, 28)

In [107]:
batch_size = 128

In [108]:
class AutoEncoder(object):
    def __init__(self):
        self.weights = {
            "encoder_h1": tf.Variable(tf.random_normal([5,5,1,16])),
            "encoded": tf.Variable(tf.random_normal([5,5,16,4])),
            "decoder_h1": tf.Variable(tf.random_normal([2,2,16,4])),
            "output": tf.Variable(tf.random_normal([2,2,1,16]))
        }
        self.biases = {
            "encoder_h1": tf.Variable(tf.zeros([16])),
            "encoded": tf.Variable(tf.zeros([4])),
            "decoder_h1": tf.Variable(tf.zeros([16])),
            "output": tf.Variable(tf.zeros([1]))
        }
    
    def forward(self,X):
        tf.disable_eager_execution()
        
        encoder_h1 = tf.nn.conv2d(X, self.weights["encoder_h1"], strides=[1,1,1,1], padding="SAME")
        encoder_h1 = tf.nn.bias_add(encoder_h1, self.biases["encoder_h1"])
        encoder_h1 = tf.nn.max_pool2d(encoder_h1, ksize = [1,2,2,1], strides = [1,2,2,1], padding="SAME")

        encoded = tf.nn.conv2d(encoder_h1, self.weights["encoded"], strides=[1,1,1,1], padding="SAME")
        encoded = tf.nn.bias_add(encoded, self.biases["encoded"])
        encoded = tf.nn.max_pool2d(encoded, ksize = [1,2,2,1], strides = [1,2,2,1], padding="SAME")
        
        decoder_h1 = tf.nn.conv2d_transpose(encoded,
                                            self.weights["decoder_h1"],
                                            output_shape = [batch_size, 14,14,16], 
                                            strides = [1,2,2,1])
        
        decoder_h1 = tf.nn.bias_add(decoder_h1, self.biases["decoder_h1"])
        
        output = tf.nn.conv2d_transpose(decoder_h1,
                                        self.weights["output"],
                                        output_shape = [batch_size, 28,28,1], 
                                        strides = [1,2,2,1])
        
        output = tf.nn.bias_add(output, self.biases["output"])
      
            
        return output, encoded



neural_net = AutoEncoder()

In [109]:
tf.disable_eager_execution()
X = tf.placeholder(tf.float32, shape=[batch_size, 28,28,1])

In [110]:
logits, encoded = neural_net.forward(X)
logits.shape

TensorShape([128, 28, 28, 1])

In [111]:
X_test.shape

(10000, 28, 28)

In [112]:
loss = tf.reduce_mean(tf.square(logits - X))

In [113]:
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

In [114]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
n_epochs = 200
with tf.Session() as sess:
    init.run()
    for epoch in tqdm_notebook(range(n_epochs)):
        prev = 0
        n_batches = len(X_train) // batch_size
        temp_loss = 0
        for i in range(n_batches):
            batch_x = X_train[prev: prev+batch_size]
            prev = prev+batch_size
            feed = {X : batch_x.reshape(-1, 28,28,1)}
            sess.run([optimizer], feed_dict = feed)
            temp_loss += loss.eval(feed_dict = feed)
        save_path = saver.save(sess, "/model/model.ckpt")
        print("Epoch: ",epoch, " Loss: ",temp_loss/n_batches)
    
    

Epoch:  0  Loss:  583600277.1623932
Epoch:  1  Loss:  60813995.307692304
Epoch:  2  Loss:  22492200.215811964
Epoch:  3  Loss:  9974092.637820512
Epoch:  4  Loss:  4826694.336004274
Epoch:  5  Loss:  2507822.8707264955
Epoch:  6  Loss:  1383830.7377136752
Epoch:  7  Loss:  792005.499732906
Epoch:  8  Loss:  453980.7000534188
Epoch:  9  Loss:  250983.94968616453
Epoch:  10  Loss:  128903.23624465812
Epoch:  11  Loss:  59643.84125434028
Epoch:  12  Loss:  25160.4767711672
Epoch:  13  Loss:  11256.451562082666
Epoch:  14  Loss:  7045.722380809295
Epoch:  15  Loss:  6098.372646233975
Epoch:  16  Loss:  5860.5859145466075
Epoch:  17  Loss:  5704.036577273638
Epoch:  18  Loss:  5540.399265908787
Epoch:  19  Loss:  5366.065884581998
Epoch:  20  Loss:  5186.206695295807
Epoch:  21  Loss:  5006.748064611712
Epoch:  22  Loss:  4832.154911399906
Epoch:  23  Loss:  4664.4691495976895
Epoch:  24  Loss:  4503.936256865151
Epoch:  25  Loss:  4348.522934090378


KeyboardInterrupt: 

In [ ]:
labels = []
with tf.Session() as sess:
    saver.restore(sess, "/model/model.ckpt")
    
    x,output = neural_net.forward(tf.cast(X_test.reshape(10000,28,28,1), tf.float32))
    output = output.eval(feed_dict = feed).reshape(-1,7*7*4)
    print(output.shape)
    cluster = AgglomerativeClustering(n_clusters=10, affinity='euclidean', linkage='ward')
    cluster.fit_predict(output)
    labels = cluster.labels_

In [ ]:
labels

In [ ]:
import matplotlib.pyplot as plt

In [120]:
for j,i in enumerate(labels):
    if i == 5:
        plt.imshow(X_test[j], cmap = "gray")